In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR

In [2]:
class Argument():
    def __init__(self, batch_size=256, test_batch_size=1000, epochs=2, lr=4.0, no_cuda=False, save_model=False):
        
        self.batch_size = batch_size
        self.test_batch_size = test_batch_size
        self.epochs = epochs
        self.lr = lr
        self.no_cuda = no_cuda
        self.save_model = save_model


In [3]:
args = Argument()
use_cuda = not args.no_cuda and torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

In [4]:
#%%time

#import data_saver
#data_saver.save_data()

In [5]:
%%time

train_data = torch.load('data/train_data.pt', map_location=device)
train_labels = torch.load('data/train_labels.pt', map_location=device)

test_data = torch.load('data/test_data.pt', map_location=device)
test_labels = torch.load('data/test_labels.pt', map_location=device)

Wall time: 1.94 s


In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(1600, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

def train(args, model, device, optimizer, scheduler):
    batch_size = args.batch_size
    model.train()
    num_batches = (len(train_data)-1)//batch_size + 1
    for i in range(num_batches):
        data = train_data[batch_size*i:batch_size*(i+1)]
        target = train_labels[batch_size*i:batch_size*(i+1)]
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        scheduler.step()
    


def test(args, model, device, epoch):
    model.eval()
    batch_size = args.test_batch_size
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i in range(len(test_data)//batch_size):
            data = test_data[batch_size*i:batch_size*(i+1)]
            target = test_labels[batch_size*i:batch_size*(i+1)]
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_data)

    print('Epoch {} Test set: Average loss: {:.4f}, Accuracy: ({:.2f}%)\n'.format(
        epoch, test_loss, 100. * correct / len(test_data)))
    return 100.*correct / len(test_data)


def main():
    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = OneCycleLR(optimizer, max_lr=args.lr, epochs=args.epochs,
                           steps_per_epoch=(len(train_data)-1)//args.batch_size + 1, cycle_momentum=False)
    
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, optimizer, scheduler)
        test_acc = test(args, model, device, epoch)

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


In [7]:
%%timeit
main()

Epoch 1 Test set: Average loss: 0.0545, Accuracy: (98.18%)

Epoch 2 Test set: Average loss: 0.0294, Accuracy: (98.94%)

Epoch 1 Test set: Average loss: 0.0544, Accuracy: (98.28%)

Epoch 2 Test set: Average loss: 0.0294, Accuracy: (98.99%)

Epoch 1 Test set: Average loss: 0.0475, Accuracy: (98.41%)

Epoch 2 Test set: Average loss: 0.0258, Accuracy: (99.16%)

Epoch 1 Test set: Average loss: 0.0650, Accuracy: (97.85%)

Epoch 2 Test set: Average loss: 0.0254, Accuracy: (99.13%)

Epoch 1 Test set: Average loss: 0.0554, Accuracy: (98.33%)

Epoch 2 Test set: Average loss: 0.0279, Accuracy: (99.05%)

Epoch 1 Test set: Average loss: 0.0597, Accuracy: (98.15%)

Epoch 2 Test set: Average loss: 0.0295, Accuracy: (99.00%)

Epoch 1 Test set: Average loss: 0.0665, Accuracy: (98.08%)

Epoch 2 Test set: Average loss: 0.0271, Accuracy: (99.05%)

Epoch 1 Test set: Average loss: 0.0729, Accuracy: (97.59%)

Epoch 2 Test set: Average loss: 0.0265, Accuracy: (99.10%)

2.31 s ± 23.2 ms per loop (mean ± std. d